In [ ]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL
# =========================================================

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Libs de Modelagem e Estatística
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.datasets import get_rdataset
from scipy.stats import friedmanchisquare
import pmdarima as pm
import scikit_posthocs as sp
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, MLP, LSTM, Autoformer, NHITS

# Libs de Avaliação
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from IPython.display import display, Markdown

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# =========================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES (SETUP E PROCESSAMENTO)
# =========================================================

In [ ]:
def definir_seed(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
def carregar_serie(nome, pasta_cache="./data/bronze"):

    nome_base = nome.lower()
    # Garante que o diretório de cache exista
    os.makedirs(pasta_cache, exist_ok=True)
    caminho_arquivo = os.path.join(pasta_cache, f"{nome_base}.csv")

    # Passo 1: Verifica se o arquivo já existe localmente
    if os.path.exists(caminho_arquivo):
        print(f"Carregando dataset '{nome}' do cache local: {caminho_arquivo}")
        # Se existir, carrega diretamente do CSV, garantindo que a data seja o índice
        df = pd.read_csv(caminho_arquivo, parse_dates=['date'], index_col='date')
        # Retorna a série com o nome original para consistência
        df['value'].name = nome
        return df['value']

    # Passo 2: Se não existir, faz o download
    print(f"Cache não encontrado. Buscando dados de '{nome}' via statsmodels...")
    
    serie = None
    if nome_base == "airpassengers":
        dados = get_rdataset("AirPassengers", package="datasets").data
        serie = pd.Series(dados['value'].values, index=pd.date_range(start="1949-01-01", periods=len(dados), freq="MS"), name=nome)
    elif nome_base == "lynx":
        dados = get_rdataset("lynx", package="datasets").data
        serie = pd.Series(dados['value'].values, index=pd.date_range(start="1821", periods=len(dados), freq="YE-DEC"), name=nome)
    elif nome_base == "co2":
        dados = get_rdataset("CO2", package="datasets").data
        dados = dados.ffill()
        serie = pd.Series(dados['value'].values, index=pd.date_range(start="1958-03-29", periods=len(dados), freq="MS"), name=nome)
    elif nome_base == "austres":
        dados = get_rdataset("austres", package="datasets").data
        serie = pd.Series(dados['value'].values, index=pd.date_range(start="1971-03-01", periods=len(dados), freq="QS-MAR"), name=nome)
    elif nome_base == "nottem":
        dados = get_rdataset("nottem", package="datasets").data
        serie = pd.Series(dados['value'].values, index=pd.date_range(start="1920-01-01", periods=len(dados), freq="MS"), name=nome)
    else:
        raise ValueError(f"Lógica de download para a série '{nome}' não implementada.")

    # Passo 3: Salva a série baixada no cache para uso futuro
    if serie is not None:
        print(f"-> Salvando cópia do dataset '{nome}' em cache: {caminho_arquivo}")
        df_para_salvar = pd.DataFrame({"date": serie.index, "value": serie.values})
        df_para_salvar.to_csv(caminho_arquivo, index=False)
    
    return serie

In [ ]:
def dividir_serie_temporal(serie, percentual_treino=0.85):
    tamanho_total = len(serie)
    ponto_corte_treino = int(tamanho_total * percentual_treino)
    treino = serie.iloc[:ponto_corte_treino]
    teste = serie.iloc[ponto_corte_treino:]
    return treino, teste

def preparar_dados_para_neuralforecast(serie, nome_serie):
    df = serie.reset_index()
    df.columns = ['ds', 'y']
    df['unique_id'] = nome_serie
    return df

In [ ]:
# =========================================================
# SEÇÃO 3: FUNÇÕES PARA CÁLCULO DE MÉTRICAS E MODELAGEM
# =========================================================

In [ ]:
def calcular_metricas(y_true, y_pred, y_train):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.inf
    n = len(y_train)
    d = np.sum(np.abs(y_train[1:] - y_train[:-1])) / (n - 1) if n > 1 else np.nan
    mase = np.mean(np.abs(y_true - y_pred)) / d if d is not np.nan and d > 0 else np.inf
    return {'RMSE': rmse, 'MAPE(%)': mape, 'MASE': mase}

In [ ]:
# =========================================================
# SEÇÃO 4: PIPELINE AVANÇADO PARA O ARIMA
# =========================================================


In [ ]:
def encontrar_melhor_arima_auto(treino_log, freq):
    """Usa auto_arima para encontrar a melhor ordem ARIMA, incluindo sazonalidade."""
    print("Buscando melhor ordem ARIMA com auto_arima...")
    m = 12 if freq.startswith('M') else (4 if freq.startswith('Q') else 1)
    auto_arima_model = pm.auto_arima(treino_log, m=m, seasonal=True, trace=False, error_action='ignore', suppress_warnings=True, stepwise=True)
    print(f"Melhor ordem encontrada: {auto_arima_model.order} Sazonal: {auto_arima_model.seasonal_order}")
    return auto_arima_model.order, auto_arima_model.seasonal_order

In [ ]:
# =========================================================
# SEÇÃO 5: PIPELINE DE EXPERIMENTO COMPLETO E AVANÇADO
# =========================================================

In [ ]:
def executar_experimento(nome_da_serie, horizonte):
    """Executa o pipeline completo, agora incluindo o Híbrido Recursive-Direct do artigo."""
    try:
        SEED = 42
        definir_seed(SEED)
        MAX_INPUT_SIZE = 24
        MAX_STEPS_NEURAL = 150
        
        serie_original = carregar_serie(nome_da_serie)
        
        percentual_treino = 1 - (horizonte / len(serie_original))
        if percentual_treino < 0.5:
             print(f"AVISO: Horizonte {horizonte} é muito grande para a série '{nome_da_serie}'. Pulando.")
             return None
        
        treino_orig, teste_orig = dividir_serie_temporal(serie_original, percentual_treino=percentual_treino)
        serie_log = np.log(serie_original)
        treino_log, _ = dividir_serie_temporal(serie_log, percentual_treino=percentual_treino)
        
        freq = serie_original.index.freqstr or pd.infer_freq(serie_original.index)
        if freq is None: return None

        previsoes_teste = {'y_true': teste_orig.values}
        
        # --- 1. Modelo ARIMA (Base para ambos os híbridos) ---
        modelo_arima = None
        try:
            print("Processando: ARIMA")
            ordem, ordem_sazonal = encontrar_melhor_arima_auto(treino_log, freq)
            modelo_arima = ARIMA(treino_log.asfreq(freq), order=ordem, seasonal_order=ordem_sazonal).fit()
            preds_log_teste_arima = modelo_arima.forecast(steps=horizonte)
            previsoes_teste['ARIMA'] = np.exp(preds_log_teste_arima).values
        except Exception as e: 
            print(f"AVISO: ARIMA falhou: {e}")
            return None # Retorna se o ARIMA falhar, pois é base para os híbridos

        # --- 2. Modelos Neurais Puros ---
        df_treino_log_nf = preparar_dados_para_neuralforecast(treino_log, nome_da_serie)
        modelos_para_testar = {'N-BEATS': NBEATS, 'MLP': MLP, 'LSTM': LSTM, 'Autoformer': Autoformer, 'NHITS': NHITS}
        
        for nome_modelo, classe_modelo in modelos_para_testar.items():
            try:
                print(f"Processando: {nome_modelo}")
                modelo_neural = [classe_modelo(input_size=min(2 * horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf = NeuralForecast(models=modelo_neural, freq=freq)
                nf.fit(df=df_treino_log_nf, verbose=False)
                previsoes_teste[nome_modelo] = np.exp(nf.predict()[classe_modelo.__name__].values)
            except Exception as e: print(f"AVISO: {nome_modelo} falhou: {e}")
        
        # Obter os resíduos do treino para os modelos híbridos
        residuos_treino_log = modelo_arima.resid
            
        # --- 3. Modelo Híbrido (MIMO) - Sua implementação original ---
        try:
            print("Processando: Híbrido (MIMO)")
            df_residuos_nf = preparar_dados_para_neuralforecast(residuos_treino_log, "residuos")
            # Treina um único modelo para prever todos os H passos
            modelo_residuos_mimo = [NBEATS(input_size=min(2*horizonte, MAX_INPUT_SIZE), h=horizonte, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
            nf_residuos_mimo = NeuralForecast(models=modelo_residuos_mimo, freq=freq)
            nf_residuos_mimo.fit(df=df_residuos_nf, verbose=False)
            preds_residuos_log_mimo = nf_residuos_mimo.predict()['NBEATS'].values
            previsoes_teste['Híbrido (MIMO)'] = previsoes_teste['ARIMA'] + preds_residuos_log_mimo
        except Exception as e: print(f"AVISO: Híbrido (MIMO) falhou: {e}")

        # =========================================================================================
        # --- 4. NOVO: Modelo Híbrido (Recursive-Direct) - Conforme o Artigo HyS-MF ---
        # =========================================================================================
        try:
            print("Processando: Híbrido (Recursive-Direct)")
            preds_residuos_log_direct = []
            
            # Loop para treinar um modelo especialista para cada passo do horizonte
            for h_step in range(1, horizonte + 1):
                print(f"  -> Treinando especialista para horizonte h={h_step}")
                
                # Prepara os dados para a abordagem Direct: o alvo 'y' é o resíduo h_step passos no futuro
                df_residuos_h = residuos_treino_log.to_frame(name='y')
                df_residuos_h['ds'] = df_residuos_h.index
                df_residuos_h['y'] = df_residuos_h['y'].shift(-h_step + 1) # Desloca o alvo
                df_residuos_h.dropna(inplace=True) # Remove os NaNs criados pelo shift
                df_residuos_h['unique_id'] = f"residuos_h{h_step}"

                # Treina um modelo N-BEATS para prever apenas 1 passo à frente (o seu alvo específico)
                modelo_residuos_direct = [NBEATS(input_size=min(2*horizonte, MAX_INPUT_SIZE), h=1, max_steps=MAX_STEPS_NEURAL, scaler_type='standard', random_seed=SEED)]
                nf_residuos_direct = NeuralForecast(models=modelo_residuos_direct, freq=freq)
                nf_residuos_direct.fit(df=df_residuos_h, verbose=False)
                
                # Faz a previsão de 1 passo (que corresponde ao h_step do resíduo)
                pred_h = nf_residuos_direct.predict()['NBEATS'].values[0]
                preds_residuos_log_direct.append(pred_h)

            # Combina a previsão do ARIMA com a soma das previsões dos resíduos
            previsoes_teste['Híbrido (Recursive-Direct)'] = previsoes_teste['ARIMA'] + np.array(preds_residuos_log_direct)
        except Exception as e: 
            print(f"AVISO: Híbrido (Recursive-Direct) falhou: {e}")

        df_final = pd.DataFrame(previsoes_teste, index=teste_orig.index)
        df_final['dataset'] = nome_da_serie
        df_final['horizonte'] = horizonte
        return df_final.reset_index().rename(columns={'index': 'ds'})
    
    except Exception as e:
        print(f"ERRO GERAL no processamento de '{nome_da_serie}' para o horizonte {horizonte}: {e}")
        return None

In [ ]:
# =========================================================
# SEÇÃO 6: ORQUESTRADOR (LÓGICA DA CAMADA SILVER)
# =========================================================

In [ ]:
# --- Configurações ---
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'austres', 'lynx']
VETOR_DE_HORIZONTES = [10, 12, 15, 24]
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

# --- Lógica de Execução Incremental ---
# Apaga o arquivo antigo para garantir uma execução limpa
if os.path.exists(output_file):
    os.remove(output_file)

header_escrito = False

In [ ]:
for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        # Executa o experimento para uma combinação de dataset e horizonte
        df_resultado_detalhado = executar_experimento(dataset, horizonte)
        
        if df_resultado_detalhado is not None:
            # Lógica para salvar incrementalmente no arquivo da camada Silver
            if not header_escrito:
                # Escreve o cabeçalho apenas na primeira vez
                df_resultado_detalhado.to_csv(output_file, index=False, mode='w', header=True)
                header_escrito = True
            else:
                # Anexa os novos resultados sem o cabeçalho
                df_resultado_detalhado.to_csv(output_file, index=False, mode='a', header=False)

print(f"\nProcesso finalizado. Resultados salvos em '{output_file}'")

In [ ]:
# =========================================================
# SEÇÃO 6: ORQUESTRADOR
# =========================================================

In [ ]:
LISTA_DE_DATASETS = ['AirPassengers', 'co2', 'nottem', 'austres', 'lynx'] 
VETOR_DE_HORIZONTES = [10, 12, 15, 24]
resultados_gerais = []
output_dir = "./data/silver"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "resultados_completos.csv")

In [ ]:
for dataset in tqdm(LISTA_DE_DATASETS, desc="Processando Datasets"):
    for horizonte in tqdm(VETOR_DE_HORIZONTES, desc=f"Testando Horizontes para {dataset}", leave=False):
        df_resultado_detalhado = executar_experimento(dataset, horizonte)
        if df_resultado_detalhado is not None:
            resultados_gerais.append(df_resultado_detalhado)

In [ ]:
if resultados_gerais:
    df_final = pd.concat(resultados_gerais)
    df_final.to_csv(output_file, index=False)
    print(f"\nArquivo '{output_file}' salvo com sucesso!")

In [ ]:
# =================================================================================
# SEÇÃO 7: GERAÇÃO DE RELATÓRIOS (ARQUITETURA MODULAR)
# =================================================================================

In [ ]:
def calcular_metricas_finais(df_results):
    """
    Função auxiliar robusta para calcular as métricas a partir do DataFrame de previsões brutas.
    """
    modelos = [col for col in df_results.columns if col not in ['ds', 'y_true', 'dataset', 'horizonte']]
    
    # Dicionário para armazenar dados de treino para o cálculo do MASE
    y_train_dict = {}
    
    # Para cada dataset, carrega os dados de treino e teste originais para o cálculo do MASE
    for dataset_nome in df_results['dataset'].unique():
        horizonte_max_ds = df_results[df_results['dataset'] == dataset_nome]['horizonte'].max()
        serie_original = carregar_serie(dataset_nome)
        percentual_treino = 1 - (horizonte_max_ds / len(serie_original))
        treino, _ = dividir_serie_temporal(serie_original, percentual_treino)
        y_train_dict[dataset_nome] = treino.values

    # Transforma o DataFrame de 'largo' para 'longo' para facilitar a agregação
    df_melted = df_results.melt(id_vars=['ds', 'y_true', 'dataset', 'horizonte'], 
                                value_vars=modelos, var_name='Modelo', value_name='y_pred')
    
    metricas_gerais = []
    # Agrupa para calcular as métricas para cada combinação de dataset, horizonte e modelo
    for (dataset, horizonte, modelo), group in tqdm(df_melted.groupby(['dataset', 'horizonte', 'Modelo']), desc="Calculando Métricas"):
        if not group['y_pred'].isnull().all():
            # Passa a fatia correta dos dados de treino para o MASE
            y_train_correto = y_train_dict[dataset]
            
            # Calcula as métricas
            metricas_dict = calcular_metricas(group['y_true'], group['y_pred'], y_train_correto)
            
            # Adiciona as informações de metadados ao dicionário
            metricas_dict['dataset'] = dataset
            metricas_dict['horizonte'] = horizonte
            metricas_dict['Modelo'] = modelo
            
            metricas_gerais.append(metricas_dict)
    
    df_metricas_final = pd.DataFrame(metricas_gerais)
    # Renomeia colunas para clareza nos relatórios
    df_metricas_final.rename(columns={'RMSE': 'Mean RMSE', 'MAPE(%)': 'Mean MAPE(%)', 'MASE': 'Mean MASE'}, inplace=True)
    return df_metricas_final

In [ ]:
def plotar_evolucao_erro(df_metricas, vetor_horizontes):
    """RELATÓRIO 1: Gera o gráfico de linha da evolução do erro."""
    print("\n--- RELATÓRIO 1: EVOLUÇÃO DO ERRO (RMSE) POR HORIZONTE ---")
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=df_metricas, x='horizonte', y='Mean RMSE', hue='Modelo', style='Modelo', markers=True, dashes=False)
    plt.title("Evolução do Erro (RMSE) com o Aumento do Horizonte", fontsize=16)
    plt.xlabel("Horizonte de Previsão"); plt.ylabel("RMSE Médio"); plt.grid(True)
    if not df_metricas.empty:
        plt.xticks(vetor_horizontes)
    plt.legend(title='Modelo'); plt.show()

In [ ]:
def exibir_desempenho_agregado(df_foco):
    """RELATÓRIO 2: Mostra a tabela de desempenho geral agregado."""
    print("\n--- RELATÓRIO 2: DESEMPENHO GERAL (MÉDIA NO HORIZONTE MAIS LONGO) ---")
    df_agrupado = df_foco.groupby('Modelo')[['Mean RMSE', 'Mean MAPE(%)', 'Mean MASE']].mean()
    display(df_agrupado.style.format('{:.3f}').highlight_min(axis=0, props='background-color: #4285F4; color: white;'))

In [ ]:
def exibir_desempenho_detalhado(df_foco):
    """RELATÓRIO 3: Mostra a tabela de desempenho detalhado por dataset."""
    print("\n--- RELATÓRIO 3: DESEMPENHO DETALHADO POR DATASET (HORIZONTE MAIS LONGO) ---")
    df_reporte_detalhado = df_foco.set_index(['dataset', 'Modelo']).drop(columns=['horizonte'])
    display(df_reporte_detalhado.style.format('{:.3f}'))

In [ ]:
def exibir_tabela_ranking(df_foco):
    """RELATÓRIO 4: Mostra a tabela de ranking e a retorna para uso posterior."""
    print("\n--- RELATÓRIO 4: RANKING DOS MODELOS (BASEADO EM RMSE, HORIZONTE MAIS LONGO) ---")
    df_rank = df_foco.copy()
    df_rank['Rank'] = df_rank.groupby('dataset')['Mean RMSE'].rank().astype(int)
    df_pivot_rank = df_rank.pivot_table(index='dataset', columns='Modelo', values='Rank')
    if len(df_pivot_rank) > 1:
        df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)
    display(df_pivot_rank.style.format('{:.1f}').highlight_min(axis=1, props='background-color: #4285F4; color: white;'))
    return df_pivot_rank

In [ ]:
def plotar_diferenca_percentual(df_foco):
    """RELATÓRIO 5: Mostra o gráfico de ganho percentual do modelo híbrido."""
    print("\n--- RELATÓRIO 5: GANHO PERCENTUAL DO MODELO HÍBRIDO (BASEADO EM MAPE, HORIZONTE MAIS LONGO) ---")
    df_pivot_mape = df_foco.pivot_table(index='dataset', columns='Modelo', values='Mean MAPE(%)')
    modelo_referencia_hibrido = 'Híbrido (MIMO)'
    if modelo_referencia_hibrido in df_pivot_mape.columns:
        mape_hibrido = df_pivot_mape[modelo_referencia_hibrido]
        df_pd = pd.DataFrame(index=df_pivot_mape.index)
        for modelo in [m for m in df_pivot_mape.columns if m != modelo_referencia_hibrido]:
            df_pd[f'Ganho sobre {modelo} (%)'] = 100 * (df_pivot_mape[modelo] - mape_hibrido) / df_pivot_mape[modelo]
        ax = df_pd.plot(kind='bar', figsize=(14, 7), grid=True, rot=45); ax.set_ylabel("Melhora Percentual (%)"); ax.set_xlabel("Dataset")
        ax.set_title(f"Diferença Percentual (PD%): Ganho de Performance do {modelo_referencia_hibrido}"); plt.tight_layout(); plt.show()

In [ ]:
def exibir_analise_estatistica_demsar(df_pivot_rank, maior_horizonte):
    """RELATÓRIO 6: Executa e exibe os testes de Friedman e Nemenyi."""
    print("\n\n" + "="*60); print(f"     RELATÓRIO 6: ANÁLISE ESTATÍSTICA GLOBAL (FRIEDMAN + NEMENYI, HORIZONTE {int(maior_horizonte)})"); print("="*60)
    
    # --- CORREÇÃO APLICADA AQUI ---
    # Inicializamos as variáveis como None para garantir que sempre existam.
    p_values_nemenyi = None
    avg_ranks = None
    
    df_rank_data = df_pivot_rank.drop('Média do Rank', errors='ignore')
    if df_rank_data.empty:
        print("AVISO: Tabela de ranking vazia, não foi possível executar a análise estatística.")
        # Retorna os valores nulos
        return p_values_nemenyi, avg_ranks

    try:
        stat, p_value = friedmanchisquare(*[df_rank_data[col].values for col in df_rank_data.columns])
        print(f"\n--- Teste de Friedman ---\np-valor: {p_value:.4f}")

        if p_value < 0.05:
            print("\n**Conclusão: Há uma diferença estatisticamente significativa entre os modelos.**")
            
            print("\n--- Teste Post-hoc de Nemenyi (p-valores par a par) ---")
            df_rank_melted = df_rank_data.reset_index().melt(id_vars='dataset', var_name='Modelo', value_name='Rank')
            p_values_nemenyi = sp.posthoc_nemenyi_friedman(df_rank_melted, melted=True, group_col='Modelo', block_col='dataset', y_col='Rank')
            display(p_values_nemenyi.style.format('{:.3f}').applymap(lambda x: 'background-color: lightgreen' if x < 0.05 else ''))

            # Calcula os ranks médios apenas se o teste for significativo
            avg_ranks = df_pivot_rank.mean(axis=0).drop('Média do Rank', errors='ignore')
        else:
            print("\n**Conclusão: Não há evidência de uma diferença estatística significativa entre os modelos.**")
            
    except Exception as e:
        print(f"AVISO: A análise estatística avançada falhou: {e}")
        
    # A função agora sempre retorna uma tupla, mesmo que os valores sejam None
    return p_values_nemenyi, avg_ranks    

In [ ]:
def plotar_diagrama_diferenca_critica(df_pivot_rank, alpha=0.05):
    """RELATÓRIO ADICIONAL: Gera o Diagrama de Diferença Crítica (CD Plot)."""
    if df_pivot_rank.empty:
        print("AVISO: Tabela de ranking vazia, não é possível gerar o CD Plot.")
        return

    print("\\n--- DIAGRAMA DE DIFERENÇA CRÍTICA (CD PLOT) ---")
    display(Markdown("Este gráfico resume o teste de Nemenyi. Modelos que **NÃO** são significativamente diferentes estão conectados por uma linha horizontal."))
    
    # Extrai os dados de ranking e os nomes dos modelos
    rank_data = df_pivot_rank.drop('Média do Rank', errors='ignore').values
    model_names = df_pivot_rank.columns
    
    # Calcula a diferença crítica (CD)
    # N = número de datasets, k = número de modelos
    N = len(rank_data)
    k = len(model_names)
    
    # Importa a tabela de valores críticos q_alpha do scikit-posthocs
    from scikit_posthocs._critical_values import get_critical_value
    q_alpha = get_critical_value(k, alpha)
    
    cd = q_alpha * np.sqrt(k * (k + 1) / (6 * N))
    
    # Calcula os ranks médios
    avg_ranks = df_pivot_rank.loc['Média do Rank'].sort_values()
    
    # Gera o gráfico usando a função de plotagem do scikit-posthocs
    sp.sign_plot(avg_ranks, cd=cd)
    plt.title(f"Diagrama de Diferença Crítica (Teste de Nemenyi, alpha={alpha})", fontsize=16)
    plt.xlabel("Ranking Médio")
    plt.show()

# Na sua função principal de relatórios, a chamada seria:
# df_pivot_rank = exibir_tabela_ranking(df_foco_maior_h)
# plotar_diagrama_diferenca_critica(df_pivot_rank)

In [ ]:
# --- NOVA FUNÇÃO DE RELATÓRIO (BASEADA NA TABELA 4 DO ARTIGO) ---
def exibir_tabela_ranking(df_foco, metrica='Mean RMSE'):
    """RELATÓRIO 4: Mostra a tabela de ranking dos modelos."""
    print(f"\n--- RELATÓRIO 4: RANKING DOS MODELOS (BASEADO EM {metrica}) ---")
    df_rank = df_foco.copy()
    rank_col_name = f'Rank_{metrica}'
    df_rank[rank_col_name] = df_rank.groupby('dataset')[metrica].rank().astype(int)
    df_pivot_rank = df_rank.pivot_table(index='dataset', columns='Modelo', values=rank_col_name)
    if len(df_pivot_rank) > 1:
        df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)
    display(df_pivot_rank.style.format('{:.1f}').highlight_min(axis=1, props='background-color: #4285F4; color: white;'))
    return df_pivot_rank

# --- NOVA FUNÇÃO DE RELATÓRIO (BASEADA NA TABELA 4 DO ARTIGO) ---
def exibir_ranking_mape_artigo(df_foco):
    """RELATÓRIO EXTRA 1: Gera a tabela de ranking por MAPE, no estilo da Tabela 4 do artigo."""
    print("\n\n" + "="*60)
    print("     RELATÓRIO ADICIONAL: TABELA DE RANKING POR MAPE (ESTILO ARTIGO)")
    print("="*60)
    
    df_rank_mape = df_foco.copy()
    df_rank_mape['Rank_MAPE'] = df_rank_mape.groupby('dataset')['Mean MAPE(%)'].rank().astype(int)
    df_pivot = df_rank_mape.pivot_table(index='Modelo', columns='dataset', values='Rank_MAPE')
    
    # Calcula a Média e Mediana do Rank para cada modelo
    df_pivot['Média'] = df_pivot.mean(axis=1)
    df_pivot['Mediana'] = df_pivot.median(axis=1)
    
    def highlight_top3(s):
        is_top3 = s <= 3
        return ['background-color: blue' if v else '' for v in is_top3]

    styled_pivot = (df_pivot.style
                    .format('{:.1f}')
                    .apply(highlight_top3, subset=pd.IndexSlice[:, [c for c in df_pivot.columns if c not in ['Média', 'Mediana']]])
                    .set_caption("Ranking dos modelos por dataset baseado no MAPE (1 = Melhor)."))
    
    display(styled_pivot)

# --- NOVA FUNÇÃO DE RELATÓRIO (BASEADA NA FIGURA 9 DO ARTIGO) ---
def plotar_pd_agregado(df_foco):
    """RELATÓRIO EXTRA 2: Gera o gráfico de Diferença Percentual (PD%) agregado."""
    print("\n\n" + "="*60)
    print("     RELATÓRIO ADICIONAL: GRÁFICO DE DIFERENÇA PERCENTUAL AGREGADO (ESTILO FIGURA 9)")
    print("="*60)

    modelo_referencia_hibrido = 'Híbrido (MIMO)'
    # Calcula o MAPE médio para cada modelo, em todos os datasets
    df_mean_mape = df_foco.groupby('Modelo')['Mean MAPE(%)'].mean()

    if modelo_referencia_hibrido in df_mean_mape.index:
        mape_hibrido = df_mean_mape[modelo_referencia_hibrido]
        pd_values = []
        for modelo, mape_medio in df_mean_mape.items():
            if modelo != modelo_referencia_hibrido:
                pd_value = 100 * (mape_medio - mape_hibrido) / mape_medio
                pd_values.append({'Modelo': modelo, 'PD(%)': pd_value})
        
        if pd_values:
            df_pd = pd.DataFrame(pd_values).sort_values(by='PD(%)')
            
            plt.figure(figsize=(14, 8))
            ax = sns.barplot(x='Modelo', y='PD(%)', data=df_pd, palette='viridis', hue='Modelo', legend=False)
            ax.set_title(f"Diferença Percentual (PD%) Agregada do {modelo_referencia_hibrido}", fontsize=16)
            ax.set_ylabel("Melhora Percentual (%)")
            ax.set_xlabel("Modelo Competidor")
            plt.grid(axis='x', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()


In [ ]:
# --- NOVA FUNÇÃO DE RELATÓRIO ---
def plotar_evolucao_ranking_mape(df_metricas, vetor_horizontes):
    """
    RELATÓRIO NOVO: Gera o gráfico de linha da evolução do Ranking Médio por horizonte,
    baseado no MAPE.
    """
    print("\n\n" + "="*60)
    print("     RELATÓRIO NOVO: EVOLUÇÃO DO RANKING MÉDIO (MAPE) POR HORIZONTE")
    print("="*60)
    
    # Cria uma cópia para trabalhar com segurança
    df_rank_mape = df_metricas.copy()
    
    # Calcula o ranking baseado em MAPE para cada dataset e horizonte
    df_rank_mape['Rank_MAPE'] = df_rank_mape.groupby(['dataset', 'horizonte'])['Mean MAPE(%)'].rank().astype(int)
    
    # Calcula o rank médio para cada modelo em cada horizonte, através de todos os datasets
    df_avg_rank = df_rank_mape.groupby(['horizonte', 'Modelo'])['Rank_MAPE'].mean().reset_index()
    
    plt.figure(figsize=(14, 8))
    ax = sns.lineplot(
        data=df_avg_rank,
        x='horizonte',
        y='Rank_MAPE',
        hue='Modelo',
        style='Modelo',
        markers=True,
        dashes=False,
        linewidth=2.5
    )
    ax.set_title("Evolução do Ranking Médio dos Modelos com o Aumento do Horizonte", fontsize=16)
    ax.set_xlabel("Horizonte de Previsão (Passos à Frente)")
    ax.set_ylabel("Ranking Médio (Menor é Melhor)")
    ax.grid(True)
    ax.set_xticks(vetor_horizontes)
    
    # Inverte o eixo Y para que o rank 1 (melhor) fique no topo
    ax.invert_yaxis()
    
    # Ajusta os ticks do eixo Y para serem inteiros
    y_ticks = np.arange(1, int(df_avg_rank['Rank_MAPE'].max()) + 2)
    ax.set_yticks(y_ticks)
    
    ax.legend(title='Modelo')
    plt.show()

In [ ]:
def gerar_suite_completa_de_relatorios(output_file, vetor_horizontes):
    """Função principal que orquestra a geração de todos os relatórios."""
    print("\n\n" + "="*60); print("     INICIANDO GERAÇÃO DA SUÍTE COMPLETA DE RELATÓRIOS"); print("="*60)
    try:
        df_results = pd.read_csv(output_file)
        df_metricas_final = calcular_metricas_finais(df_results)
        
        # Chamada para os relatórios descritivos
        plotar_evolucao_erro(df_metricas_final, vetor_horizontes)
        plotar_evolucao_ranking_mape(df_metricas_final, vetor_horizontes)
        
        maior_horizonte = df_metricas_final['horizonte'].max()
        df_foco_maior_h = df_metricas_final[df_metricas_final['horizonte'] == maior_horizonte]
        display(Markdown(f"### Análises Detalhadas para o Horizonte Mais Longo ({int(maior_horizonte)} passos)"))
        
        exibir_desempenho_agregado(df_foco_maior_h)
        exibir_desempenho_detalhado(df_foco_maior_h)
        df_pivot_rank = exibir_tabela_ranking(df_foco_maior_h)
        plotar_diferenca_percentual(df_foco_maior_h)
        # --- CHAMADA PARA OS NOVOS RELATÓRIOS ---
        exibir_ranking_mape_artigo(df_foco_maior_h)
        plotar_pd_agregado(df_foco_maior_h)
        
        # Chamada para a nova análise estatística
        p_values_nemenyi, avg_ranks = exibir_analise_estatistica_demsar(df_pivot_rank, maior_horizonte)
        
        # Chamada para o novo gráfico
        plotar_diagrama_diferenca_critica(p_values_nemenyi, avg_ranks)

    except FileNotFoundError:
        print(f"\nERRO: Arquivo '{output_file}' não encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao gerar os relatórios: {e}")

In [ ]:
# =================================================================================
# SEÇÃO 8: EXECUÇÃO DA GERAÇÃO DE RELATÓRIOS
# =================================================================================

In [ ]:
VETOR_DE_HORIZONTES = [10, 12, 15, 24] # Defina os horizontes que foram testados
gerar_suite_completa_de_relatorios(output_file, VETOR_DE_HORIZONTES)